## Problem 3

#### Please provide your code where noted

In [1]:
!pip install azure-storage-blob==1.5.0

In [2]:
!python example_azure_blob_download_student-2.py

sarthak_laptop\sarth
---started at :
2022-04-11 14:15:39.796457

List blobs in the container
	 Blob name: High_data_public.csv
	 Blob name: meter-readings-small.csv
	 Blob name: plugshare-small.csv
	 Blob name: true-sentiments.csv

Sample finished downloading. The application will exit.
---completed at :
2022-04-11 14:15:40.295894


In [3]:
!python example_azure_blob_download_student-2.py

sarthak_laptop\sarth
---started at :
2022-04-11 14:16:20.041733

List blobs in the container
	 Blob name: High_data_public.csv
	 Blob name: meter-readings-small.csv
	 Blob name: plugshare-small.csv
	 Blob name: true-sentiments.csv

Sample finished downloading. The application will exit.
---completed at :
2022-04-11 14:16:20.316049


In [4]:
!pip install --upgrade "ibm-watson>=4.7.1"

In [1]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EntitiesOptions, KeywordsOptions
import pandas as pd

In [2]:
dataset = pd.read_csv('plugshare-small.csv')

#### Create a variable named "review_data" and save the dataset to this variable
#### Write a line of code that removes the data where "Review" value is Nan

In [3]:
review_data = dataset.dropna(subset=["Review"])
review_data.reset_index(drop=True, inplace=True)

#### Write a line of code that shows how many observations are left within the data after removing the Nan "Review" values

In [4]:
len(review_data)

6000

#### You need to have 6000 observations after removing Nan "Review" values

#### Fill in your API key and URL and run the cell below
#### The code below is made to extract sentiment and score for you

In [5]:
# Create sentiment extractor
def get_sentiment(input):
    authenticator = IAMAuthenticator('1TeVrPCJxGHcGSxsfgq2r9deM60YH2dhoDcEq5dTS5jC')
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2020-08-01',
        authenticator=authenticator
    )
    
    natural_language_understanding.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api')
                                                   
    response = natural_language_understanding.analyze(
        text = input,
        language = 'en',
        features=Features(sentiment=SentimentOptions())).get_result()
    
    sentiment = response['sentiment']['document']['label']
    score = response['sentiment']['document']['score']

    return [sentiment, score]

#### Running the cell below will create a csv file of the result
#### Allow up to one hour to run 

In [ ]:
# Loop through all rows, and pass each review to Watson API.
review_list = []
print("- Passing reviews to Watson API..")
print("- This may take some time as we have thousands of API calls.")
i = 0
for review in review_data['Review']:
    try:
        print(i)
        i = i + 1
        review_list.append(get_sentiment(review))
    except:
        review_list.append(['error', 'error'])

        
# Combine the output from Watson into our original Dataframe
review_data['Output'] = pd.Series(review_list).values


# UDF to help split [Sentiment, Output] into 2 separate columns
def split_output(df):
    def get_rating(row):
        return row['Output'][0]
    def get_score(row):
        return row['Output'][1]

    df['Rating'] = df.apply(get_rating, axis=1)
    df['Score'] = df.apply(get_score, axis=1)
    df = df.drop('Output', 1)

    return df

review_data = split_output(review_data)


# Save our dataframe into csv
review_data.to_csv('plugshare-processed-sentiment.csv', index=False)
print("- Output successfully saved!")

- Passing reviews to Watson API..
- This may take some time as we have thousands of API calls.
0
1
2
3
4
